# 使用sklearn中的RandomForest和sklearn中的Extra_trees

## 目录
* 随机森林与Extra_Trees的简要介绍
* 随机森林中的参数
* 构建随机森林分类器
    * 使用score(X_test,y_test)评估性能
    * 使用.oob_score_评估性能
    * 使用cross_val_score(reg,X,y，cv=10)评估性能
* 随机森林中的重要属性
* 随机森林中的重要方法
* 构建随机森林回归器
* 构建Extra_Trees

## 随机森林与Extra_Trees的简要介绍
随机森林：Bagging+DecisionTree  
Extra_Trees:Bagging+DecisionTree

    
    
**对比随机森林和Extra_Trees**  
随机森林：决策树在节点划分上，在随机特征子集上寻找最优划分特征   
    * 传统的随机森林是利用袋装法的规则，平均或少数服从多数来决定集成的结果，
    而sklearn中的随机森林是平均每个样本对应的predict_proba返回的概率，得到一个平均概率，从而决定测试样本的分类。

Extra_Tree：决策树在节点划分上，使用随机的特征和随机的阈值。  
    
    优势：
    * 提供额外的随机性,抑制过拟合
    * 训练速度更快
    劣势
    * 增大了bias

## 随机森林中的参数

|参数名称|集成的弱评估器|集成算法本身|应用中的其他过程|影响程度|含义|
|:--|:--|:--|:--|:--|:--|
|criterion| √ |-|-|看具体情况|不纯度的衡量指标<br>默认为"gini"基尼系数<br>"entropy"使用信息增益|
|max_depth|√ |-|-|★★★|-|
|min_samples_leaf|√|-|-|★★|-|
|min_samples_split|√|-|-|★★|-|
|max_features|√|-|-|★|限制分枝时考虑的特征个数|
|min_impurity_decrease|√|-|-|-|限制信息增益的大小，信息增益小于设定数值的分枝不会发生|
|n_estimators|-|√|-|★★★★|-|
|oob_score|-|-|√|-|布尔值<br>默认为False<br>在建树时，是否使用袋外样本来预测模型的泛化精确性|

## 构建随机森林
使用红酒数据集

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
wine = load_wine()
X = wine.data
y = wine.target
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [2]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(
    n_estimators = 50
)
clf.fit(X_train,y_train)

/Users/zhongpingszp/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

**使用X_test,y_test来评估随机森林的准确率**

In [3]:
clf.score(X_test,y_test)

0.9777777777777777

**使用交叉验证来评估随机森林的准确率**  
交叉验证：是数据集划分为n份，依次取每一份做测试集，每n-1份做训练集，多次训练模型以观测模型稳定性的方法

In [4]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf,
                X,#在进行交叉验证时，不再需要使用划分好的数据集
                y,
                cv=10
               )

array([1.        , 0.94444444, 1.        , 0.94444444, 1.        ,
       1.        , 1.        , 1.        , 1.        , 1.        ])

In [5]:
cross_val_score(clf,X,y,cv=10).mean()

0.9669590643274854

**使用袋外数据集来评估随机森林的准确率**  
注意：当样本量或者n_estimators比较小的时候，很可能没有数据掉落在袋外，自然就无法使用obb(out of bag)数据来测试啦  
实例化时：oob_score = True  
查看再袋外数据上测试的结果：oob_score_

In [6]:
clf = RandomForestClassifier(
    n_estimators=100,
    oob_score=True
)
clf.fit(X,y)#直接用全部数据集
clf.oob_score_

0.9831460674157303

## 随机森林中的重要属性
|属性名|功能|
|:--|:--|
|.estimators_|查看子分类器|
|.oob_score_|查看在袋外数据中的准确率<br>需要配合oob_score=True使用|
|.feature_importances_|查看模型重要度<br>一般是这个特征在多次分枝中，产生的信息增益的总和，也被称为"基尼重要性"|

In [7]:
clf.feature_importances_

array([0.1233299 , 0.02794477, 0.01475516, 0.03196134, 0.02141247,
       0.04346432, 0.16554113, 0.00943992, 0.02684508, 0.14844283,
       0.08650001, 0.12023143, 0.18013166])

In [8]:
[*zip(wine.feature_names,clf.feature_importances_)]

[('alcohol', 0.1233299027835681),
 ('malic_acid', 0.027944768891321203),
 ('ash', 0.01475516141605769),
 ('alcalinity_of_ash', 0.03196133967898154),
 ('magnesium', 0.021412471184389704),
 ('total_phenols', 0.043464317752416874),
 ('flavanoids', 0.1655411276495562),
 ('nonflavanoid_phenols', 0.009439919049083843),
 ('proanthocyanins', 0.02684507671810807),
 ('color_intensity', 0.148442825102984),
 ('hue', 0.08650000544297137),
 ('od280/od315_of_diluted_wines', 0.12023142668961773),
 ('proline', 0.18013165764094372)]

## 随机森林中的重要方法
|方法名称|功能|
|:--|:--|
|apply|输入样本点，返回每个样本被分到的叶节点的索引|
|fit||
|predict||
|score||
|predict_proba|返回每个测试样本对应的被分到每一类标签的概率,有几个分类就返回几个概率值<br>如果是二分类问题，则predict_proba返回的数值大与0.5为1，小于0.5为0|

## 构建随机森林回归器

使用boston房价数据集

In [9]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

boston = load_boston()
regressor = RandomForestRegressor(n_estimators = 100)
cross_val_score(regressor,boston.data,boston.target,cv=10,
               scoring = "neg_mean_squared_error"#默认采用R2,若想采用MSE，修改scoring这个参数
               )

array([-10.79860471,  -5.15613575,  -4.88819343, -22.45832427,
       -10.76124929, -19.748457  ,  -7.0229232 , -96.91487294,
       -25.73007368, -15.22915182])

如果忘记了评测指标对应的参数的名，可以通过sklearn中metrics.SCORERS来查看

In [10]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'log_loss',
 'mean_absolute_error',
 'mean_squared_error',
 'median_absolute_error',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

## 构建Extra_Trees

In [11]:
from sklearn.ensemble import ExtraTreesClassifier

et_clf = ExtraTreesClassifier(n_estimators=500,bootstrap=True,oob_score=True)
et_clf.fit(X,y)

ExtraTreesClassifier(bootstrap=True, class_weight=None, criterion='gini',
           max_depth=None, max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [12]:
et_clf.oob_score_

0.9887640449438202